# Union of K intervals


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [296]:
class union_interval:
    def __init__(self, k = 3, epsilon = 0.1, gamma = 0.1, space = [-5,5], noise = 0, const_w = False):
        self.intervals = np.sort(np.random.uniform(space[0] + 0.0001,space[1],2*k)).reshape(k,2)#intervals always smaller than sample space
        self.ep = epsilon
        self.gamma = gamma
        self.space = space #[space[0] + 0.0001, space[1] - 0.001]#ensure bu
        self.noise = noise
        self.const_w = const_w
        self.k = k
        self.examples = np.array((space[0] - 1, 0, 0)).reshape(1,-1)#will always be ignored

    def add_example(self, ex):
        if ex.ndim == 1:#1d array
            self.examples = np.vstack((self.examples, ex))
        elif ex.ndim == 2:#2d array
            self.examples = np.concatenate((self.examples, ex), axis =0)#needs to be list
            
    def bin_search(intervals, i, k = 0):
        j = len(intervals)
        if j <= 1:
            return k
        if i < intervals[j//2]:
            return bin_search(intervals[:j//2], i, k)
        else:
            return bin_search(intervals[j//2:], i, k + j//2)
        
    def in_intervals(intervals, i):
        indx = bin_search(intervals[:,0], i)
        return intervals[indx, 0] <= i <= intervals[indx, 1]
        
    def sample(self, m = 1, samp = None, weight = None, label = None):
        if m == 1:
            samp = samp or np.random.uniform(self.space[0], self.space[1])
            weight = float(weight or self.const_w or np.random.uniform(0,1))#wieghts random between 0 and 1
            label = label or in_intervals(self.intervals, samp)
            self.add_example(np.array((samp, weight, label)))
        else:
            samples = np.random.uniform(*self.space, m)
            weights = np.random.uniform(int(self.const_w), 1, m)
            labels = np.array([in_intervals(self.intervals, i) for i in samples])
            self.add_example(np.column_stack((samples, weights, labels)))            
            
    def estimate_intervals(self):
        self.examples.view('i8,i8,i8').sort(order=['f1'], axis=0)
        made_intervals = [self.examples[:-1,2] != self.examples[1:,2]]#compare labels
        startstop = [0] + [i if i %2 == 1 else 0 for i in np.cumsum(made_intervals)]
        _, groups = np.unique(startstop, return_inverse = True)#need to exclude 0 group, need to leave in for spaceing
        num_vals = max(groups)
        indicies = [[] for i in range(num_vals+1)]#list of list of intervals locs [most specific]
        for counter, val in enumerate(groups):
            indicies[val].append(counter)#appends row index of interval at its group
        del indicies[0]
        weights = [0]*len(indicies)
        for cnter, lst in enumerate(indicies):
            weights[cnter] = np.mean(self.examples[lst, 1])
        final_indxs = [j for i,j in sorted(zip(weights, indicies), key = lambda x: x[1])][:self.k]
        final_intervals = [0]*self.k
        for cnt, lst in enumerate(final_indxs):
            if len(lst) == 1:
                final_intervals[cnt] = [self.examples[lst[0],0]]
            else: 
                final_intervals[cnt] = list((self.examples[lst[0],0], self.examples[lst[-1],0]))#+ self.examples[lst,0][-1]#get's first and last value in index
        return final_intervals

In [297]:
test1 = union_interval()
test1.sample(m = 100)
print(test1.estimate_intervals())
print(test1.intervals)

[[1.2619802606701143, 3.4355794916955205], [4.243159826371578], [1.8411280026637487]]
[[-3.58729917 -2.46776037]
 [ 1.03288572  2.47772575]
 [ 2.89398459  4.63587809]]
